# Connection using credentials set with _aws configure_

In this case you just need to call _aws configure_ in console, and set the credentials of the role you want to use to connect. Then, boto3 function calls will use these credentials by default.

In [ ]:
import boto3
import pandas as pd
import time
from io import StringIO

In [ ]:
client = boto3.client('athena')
query = "SELECT * FROM sampledb.elb_logs LIMIT 10"

response = client.start_query_execution(
    QueryString=query,
    ResultConfiguration={
        'OutputLocation': 's3://aws-athena-query-results-920137764546-us-east-1/'
    }
)

In [ ]:
# as we want to read the query result, we need to wait till it ends
while True:
    state = client.get_query_execution(QueryExecutionId = response['QueryExecutionId'])['QueryExecution']['Status']['State']
    if state == 'SUCCEEDED' or state == 'FAILED':
        break
    time.sleep(1) # let's wait one more second

# query finished, so we can now load its result
s3 = boto3.resource('s3')
bucket = 'aws-athena-query-results-920137764546-us-east-1'
obj = s3.Object(bucket, "{}.csv".format(response['QueryExecutionId']))
csv_content = obj.get()['Body'].read().decode('utf-8')

query_result = pd.read_csv(StringIO(csv_content))

In [ ]:
query_result

# Connection using credentials provided for a session

In this case you need to provide aws role credentials when you construct a client object.

In [ ]:
import boto3
import pandas as pd
import time
from io import StringIO

In [ ]:
access_key = '<your_access_key>'
secret_access_key = '<your_secret_access_key>'
region = 'us-east-1'

In [ ]:
client = boto3.client('athena',
    aws_access_key_id=access_key, # as we create a new boto3 connection, we need to specify credentials again
    aws_secret_access_key=secret_access_key,
    region_name=region 
)
query = "SELECT * FROM sampledb.elb_logs LIMIT 10"

response = client.start_query_execution(
    QueryString=query,
    ResultConfiguration={
        'OutputLocation': 's3://aws-athena-query-results-920137764546-us-east-1/'
    }
)

In [ ]:
# as we want to read the query result, we need to wait till it ends
while True:
    state = client.get_query_execution(QueryExecutionId = response['QueryExecutionId'])['QueryExecution']['Status']['State']
    if state == 'SUCCEEDED' or state == 'FAILED':
        break
    time.sleep(1) # let's wait one more second

# query finished, so we can now load its result
s3 = boto3.resource('s3',
    aws_access_key_id=access_key, # as we create a new boto3 connection, we need to specify credentials again
    aws_secret_access_key=secret_access_key,
    region_name=region
)
bucket = 'aws-athena-query-results-920137764546-us-east-1'
obj = s3.Object(bucket, "{}.csv".format(response['QueryExecutionId']))
csv_content = obj.get()['Body'].read().decode('utf-8')

query_result = pd.read_csv(StringIO(csv_content))

In [ ]:
query_result

# Other possibilities

You may check them [here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html).